In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cp -r /content/gdrive/MyDrive/PGSS2022/PGSS2023/Week03/kerasTeachableMachineModel/ /content/

In [3]:
!cp /content/gdrive/MyDrive/PGSS2022/PGSS2023/Week03/MaskData/with_mask/101-with-mask.jpg .

In [4]:
!cp /content/gdrive/MyDrive/PGSS2022/PGSS2023/Week03/MaskData/without_mask/105.jpg .

In [6]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("/content/kerasTeachableMachineModel/keras_model.h5", compile=False)

# Load the labels
class_names = open("/content/kerasTeachableMachineModel/labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
# image = Image.open("/content/101-with-mask.jpg").convert("RGB")
image = Image.open("/content/105.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


1/1 [==============================] - 1s 947ms/step
Class: NoMask
Confidence Score: 1.0


In [7]:
# Explore the model to see what its architecture is

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 1280)              410208    
                                                                 
 sequential_5 (Sequential)   (None, 2)                 128300    
                                                                 
Total params: 538,508
Trainable params: 524,428
Non-trainable params: 14,080
_________________________________________________________________


# Develop an inference SCRIPT that can run this model as an encapsulated FUNCTION

In [12]:
%%writefile runinference_keras_maskClassifier.py
# Here's the code refactored into a class with methods for loading the model and making predictions,
# and a main function that creates an instance of the class and calls its methods:

import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps


class KerasImageClassifier:
    def __init__(self, model_path, labels_path, image_path):
        # Disable scientific notation for clarity
        np.set_printoptions(suppress=True)

        self.model_path = model_path
        self.labels_path = labels_path
        self.image_path = image_path

        self.model = None
        self.class_names = None
        self.data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    def load_model_and_labels(self):
        # Load the model
        self.model = load_model(self.model_path, compile=False)
        # Load the labels
        self.class_names = open(self.labels_path, "r").readlines()

    def predict_image_class(self):
        # Open the image file
        image = Image.open(self.image_path).convert("RGB")

        # Resizing the image to be at least 224x224 and then cropping from the center
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

        # Turn the image into a numpy array
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Load the image into the array
        self.data[0] = normalized_image_array

        # Predicts the model
        prediction = self.model.predict(self.data)
        index = np.argmax(prediction)
        class_name = self.class_names[index]
        confidence_score = prediction[0][index]

        return class_name, confidence_score


"""
def main():
    model_path = "kerasTeachableMachineModel/keras_model.h5"
    labels_path = "kerasTeachableMachineModel/labels.txt"
    image_path = "105.jpg"

    classifier = KerasImageClassifier(model_path, labels_path, image_path)
    classifier.load_model_and_labels()
    class_name, confidence_score = classifier.predict_image_class()

    # Print prediction and confidence score
    print("Class:", class_name[2:], end="")
    print("Confidence Score:", confidence_score)
"""

import argparse

def main():
    # Create argument parser
    parser = argparse.ArgumentParser(description='Image Classifier')
    parser.add_argument('image', help='Path to the image that needs to be processed')

    # Parse arguments
    args = parser.parse_args()

    model_path = "kerasTeachableMachineModel/keras_model.h5"
    labels_path = "kerasTeachableMachineModel/labels.txt"
    image_path = args.image

    classifier = KerasImageClassifier(model_path, labels_path, image_path)
    classifier.load_model_and_labels()
    class_name, confidence_score = classifier.predict_image_class()

    # Print prediction and confidence score
    print("Class:", class_name[2:], end="")
    print("Confidence Score:", confidence_score)


if __name__ == "__main__":
    main()



Overwriting runinference_keras_maskClassifier.py


## Call the script that we just wrote and see what its results are when called on the command line

In [13]:
! python /content/runinference_keras_maskClassifier.py "105.jpg"

2023-07-28 19:29:53.851282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 2s 2s/step
Class: NoMask
Confidence Score: 1.0


In [14]:
! python /content/runinference_keras_maskClassifier.py "/content/101-with-mask.jpg"

2023-07-28 19:30:15.211887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 1s 986ms/step
Class: Mask
Confidence Score: 1.0


# Work out what the requirements of the colab machine are to run this model

In [8]:
! pip freeze > requirements.txt

In [ ]:
!pip install pipreqs

In [16]:
! cp /content/runinference_keras_maskClassifier.py  /content/CleanCode

In [17]:
! pipreqs /content/CleanCode

Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in /content/CleanCode/requirements.txt


In [ ]:
# Build a streamlit app to expose the trained image based classification model